### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [172]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_and_student_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_and_student_df





,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [173]:
# ----------------------------

##### totals calculations
#groups the data by type, turning it into a groupby object
schooldata_typegroup=school_and_student_df.groupby("type")
#Returns unique students and school names for each type
dist_totals=pd.DataFrame(schooldata_typegroup["school_name","Student ID",].nunique())
#turns the original school data into a grouby by object in the type series
totalbudget_dist=school_data.groupby("type").sum()
## add the total budget from the totalbudget_dist
dist_totals.join(totalbudget_dist["budget"])

###### averages calculation
#joins the data on "type", calculating the mean
dist_averages=pd.DataFrame(school_and_student_df.groupby("type").mean())
# i would like to figure out how to do this on 1 line
del dist_averages['budget']
del dist_averages['Student ID']
del dist_averages['School ID']
del dist_averages['size']



###calculates the percentages--(I could iterate through the reading and math sections of the below code to reduce redundancy)
#calculating math percentage
#pulls in the student id and type if the studen's mathscore is equal to or above
passingmath_dist = school_and_student_df.loc[school_and_student_df["math_score"]>= 70,["Student ID","type"]]
# groups the data by type then turns it into a data frame
passingmath_summary_dist=passingmath_dist.groupby("type").count()
pd.DataFrame(passingmath_summary_dist)
#Calculates the percentage of students passing math and renames the student id column correctly
passingmath_summary_dist["% Passing math"]=passingmath_summary_dist["Student ID"]/dist_totals["Student ID"]
passingmath_summary_dist=passingmath_summary_dist.rename(columns={'Student ID':'Students Passing Math','% Passing math':'% Passing Math'})
#-----
##calculating reading percentage
#pulls in the student id and type if the studen's readingscore is equal to or above 70
passingread_dist = school_and_student_df.loc[school_and_student_df["reading_score"]>= 70,["Student ID","type"]]
# groups the data by type then turns it into a data frame
passingread_summary_dist=passingread_dist.groupby("type").count()
pd.DataFrame(passingread_summary_dist)
#calculates the percentage of students passing reading and renames the student id column correctly
passingread_summary_dist["% Passing read"]=passingread_summary_dist["Student ID"] / dist_totals["Student ID"]
passingread_summary_dist=passingread_summary_dist.rename(columns={'Student ID':'Students Passing Reading','% Passing read':'% Passing Reading'})
#-----
##calculating the overall pass rates
percentpassingoverall_dist=school_and_student_df.loc[((school_and_student_df['math_score']>=70) & (school_and_student_df['reading_score']>=70)),["Student ID","type"]]

# groups the data by type then turns it into a data frame
percentpassingoverall_summary_dist=percentpassingoverall_dist.groupby("type").count()
pd.DataFrame(percentpassingoverall_summary_dist)
#calculates the percentage of students passing math/reading and renames the student id column correctly
percentpassingoverall_summary_dist["% Passing Overall"]=percentpassingoverall_summary_dist["Student ID"] / dist_totals["Student ID"]
percentpassingoverall_summary_dist=percentpassingoverall_summary_dist.rename(columns={'Student ID':'# Passing overall'})
percentpassingoverall_summary_dist.head()

#-----------------------------------------------
#need to format the table so it looks pretty
#renames the headers of dist_totals
dist_totals=dist_totals.rename(columns={'school_name':'# of schools','Student ID':'# of students'})
#renames the headers of dist_averages
dist_averages=dist_averages.rename(columns={'reading_score':'Average Reading Score','math_score':'Average Math Score'})


#list of gathered data frames excluding dist_totals
summarydf_list = [dist_averages,passingmath_summary_dist,passingread_summary_dist,percentpassingoverall_summary_dist]

#adds dist_totals to the summary dataframe manually
summary_dist_df=dist_totals
#loops through the remaining dataframes and joins them to the summary dataframe
for sublist in summarydf_list:
    summary_dist_df=pd.merge(sublist,summary_dist_df,on="type")


summary_dist_df

# need to format the columns here





,# Passing overall,% Passing Overall,Students Passing Reading,% Passing Reading,Students Passing Math,% Passing Math,Average Reading Score,Average Math Score,# of schools,# of students
type,,,,,,,,,,
Charter,11043,0.905609,11785,0.966459,11426,0.937018,83.902821,83.406183,8,12194
District,14485,0.536959,21825,0.809052,17944,0.665184,80.962485,76.987026,7,26976


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [174]:
# School Name -
# School Type -
# Total Students -
# Total School Budget -
# Per Student Budget -
# Average Math Score -
# Average Reading Score -
# % Passing Math-
# % Passing Reading-
# % Overall Passing (The percentage of students that passed math and reading.)-

### total students
#pulls in the student per school into the summary dataframe
sch_students=school_and_student_df.groupby('school_name').count()
sch_summary_df=pd.DataFrame(sch_students['student_name'])
#renames the header to Total Students
sch_summary_df = sch_summary_df.rename(columns={'student_name':'Total Students'})

##school type
# - this code pulls in the type and budget column from the original school data
sch_summary_df=pd.merge(sch_summary_df,school_data[['school_name','type','budget']],on="school_name",how="left")
#resets the index, previous line ---- maybe I could use ignore_index=True in the previous line to eliminate the need for this line
sch_summary_df=sch_summary_df.set_index('school_name')

#### budget per student
sch_summary_df["Budget Per Student"]=sch_summary_df["budget"]/sch_summary_df["Total Students"]

####calculates the averages for all series
sch_avgs_gbo = school_and_student_df.groupby("school_name").mean()
# sch_avgs_gbo.head()
#pulls in the average reading score to the summary df
sch_summary_df['Average Reading Score']=sch_avgs_gbo['reading_score']
#pulls in the average math score to the summary df
sch_summary_df['Average Math Score']=sch_avgs_gbo['math_score']


### pulls in the number of students passing reading to the summary df
sch_passing_reading = school_and_student_df.loc[school_and_student_df['reading_score']>=70,['school_name','Student ID']]
sch_passing_reading_gbo=sch_passing_reading.groupby("school_name").count()
sch_summary_df=pd.merge(sch_summary_df,sch_passing_reading_gbo,how='outer',on="school_name")
sch_summary_df=sch_summary_df.rename(columns={"Student ID":"Students Passing Reading"})

### pulls in the number of stuents passing math to the summary df
sch_passing_math=school_and_student_df.loc[school_and_student_df['math_score']>=70,['school_name','Student ID']]
sch_passing_math_gbo=sch_passing_math.groupby("school_name").count()
sch_summary_df=pd.merge(sch_summary_df,sch_passing_math_gbo,how='outer',on="school_name")
sch_summary_df=sch_summary_df.rename(columns={"Student ID":"Students Passing Math"})

### pulls in the number of students passing math and reading 
sch_passing_both=school_and_student_df.loc[((school_and_student_df['math_score']>=70) & (school_and_student_df['reading_score']>=70)) ,['school_name','Student ID']]
sch_passing_both_gbo=sch_passing_both.groupby("school_name").count()
sch_summary_df=pd.merge(sch_summary_df,sch_passing_both_gbo,how='outer',on="school_name")
sch_summary_df=sch_summary_df.rename(columns={"Student ID":"Students Passing Both"})

######calculates column for percentages (math,reading,overall) 
sch_percent_listy_loop=['Reading','Math','Both']
for category in sch_percent_listy_loop:
    sch_summary_df["% Passing "+category]=sch_summary_df["Students Passing "+category]/sch_summary_df["Total Students"]

###removes the # of students passing math reading and both since it isn't requested, just needed it for the % calcs
sch_del_list=[]
for category in sch_percent_listy_loop:
    sch_del_list.append("Students Passing "+category)

for deleter in sch_del_list:
    del sch_summary_df[deleter]

##-----------------------------------------------------------------------------------------


################!!!!!!!!!!!!!!!!    FINISH CLEAN UP !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
################
################
################
################
### i got all the data into the sch_summary_df , It could def be cleaned up a bit:
#reorder columns to match order of the readme
# change the 3 % columns to percentage format
# change the budget and budget per student to a currency format


#makes the lower case titles into titlecase, could have used the titlecase function
sch_summary_df=sch_summary_df.rename(columns={'type':'Type','budget':'Budget'})



sch_summary_df


 

,Total Students,Type,Budget,Budget Per Student,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Both
school_name,,,,,,,,,
Bailey High School,4976,District,3124928,628.0,81.033963,77.048432,0.819333,0.666801,0.546423
Cabrera High School,1858,Charter,1081356,582.0,83.975780,83.061895,0.970398,0.941335,0.913348
Figueroa High School,2949,District,1884411,639.0,81.158020,76.711767,0.807392,0.659885,0.532045
Ford High School,2739,District,1763916,644.0,80.746258,77.102592,0.792990,0.683096,0.542899
Griffin High School,1468,Charter,917500,625.0,83.816757,83.351499,0.971390,0.933924,0.905995
Hernandez High School,4635,District,3022020,652.0,80.934412,77.289752,0.808630,0.667530,0.535275
Holden High School,427,Charter,248087,581.0,83.814988,83.803279,0.962529,0.925059,0.892272
Huang High School,2917,District,1910635,655.0,81.182722,76.629414,0.813164,0.656839,0.535139
Johnson High School,4761,District,3094650,650.0,80.966394,77.072464,0.812224,0.660576,0.535392


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [175]:
top_schools_df=sch_summary_df.nlargest(5,'% Passing Both')
top_schools_df.sort_values(by=['% Passing Both'],inplace=True,ascending=False)
top_schools_df

,Total Students,Type,Budget,Budget Per Student,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Both
school_name,,,,,,,,,
Cabrera High School,1858,Charter,1081356,582.0,83.975780,83.061895,0.970398,0.941335,0.913348
Thomas High School,1635,Charter,1043130,638.0,83.848930,83.418349,0.973089,0.932722,0.909480
Griffin High School,1468,Charter,917500,625.0,83.816757,83.351499,0.971390,0.933924,0.905995
Wilson High School,2283,Charter,1319574,578.0,83.989488,83.274201,0.965396,0.938677,0.905826
Pena High School,962,Charter,585858,609.0,84.044699,83.839917,0.959459,0.945946,0.905405


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type